In [1]:
import numpy as np
import random

from tools import read_fasta_file

In [2]:
def calc_profile_matrix(dna_strings):
    """
    Given a list of DNA strings, build a profile matrix by counting the number of occurrences of each symbol at each position.
    
    DNA Strings:
    A T C C A G C T
    G G G C A A C T
    A T G G A T C T
    A A G C A A C C
    T T G G A A C T
    A T G C C A T T
    A T G G C A C T
    
    Profile:
A   5 1 0 0 5 5 0 0
C   0 0 1 4 2 0 6 1
G   1 1 6 3 0 1 0 0
T   1 5 0 0 0 1 1 6

    Args:
        + dna_strings (list of str): DNA strings to be analyzed
    
    Returns:
        profile (np.ndarray): profile matrix as a numpy.ndarray
    """
    n = len(dna_strings[0])
    profile = np.zeros(shape=(4,n), dtype=np.int64)
    
    for s in dna_strings:
        s = s.upper()
        for pos in range(n):
            if s[pos] == 'A':
                row_id = 0
            elif s[pos] == 'C':
                row_id = 1
            elif s[pos] == 'G':
                row_id = 2
            elif s[pos] == 'T':
                row_id = 3
            profile[row_id, pos] += 1
    
    return profile

In [3]:
def calc_consensus_string(profile):
    """
    Calculate consensus string of the given profile matrix.
    If several possible consensus strings exist, return any one of them, chosen randomly,
    in order not to introduce bias towards any of nucleotide.
    
    Args:
        + profile (numpy.ndarray) profile matrix
    
    Returns:
        (str) consensus string
    """
    consensus = str()
    for pos in range(profile.shape[1]):
        profile_column = profile[:, pos]
        max_count = max(profile_column)
        # If maximum count occurs just for one row (nucleotide), select corresponding nucleotide. 
        # Otherwise, randomly pick a nucleotide out of those having maximum count.
        row_indices_of_max_count = np.argwhere(profile_column == max_count).flatten()
        if len(row_indices_of_max_count) == 1:
            selected_row = row_indices_of_max_count[0]
        else:
            selected_row = random.choice(row_indices_of_max_count)
        consensus += ['A', 'C', 'G', 'T'][selected_row]
    return consensus

In [4]:
def solve(path):
    fasta = read_fasta_file(path)
    dna_strings = list(fasta.values())
    profile = calc_profile_matrix(dna_strings)
    consensus = calc_consensus_string(profile)
    print(consensus)
    for i in range(4):
        nucleotide = ['A', 'C', 'G', 'T'][i]
        row_profile = ' '.join([str(count) for count in profile[i, :]]  )
        print("{0}: {1}".format(nucleotide, row_profile))

In [5]:
# Test
solve('./txt/rosalind_cons_test.txt')

ATGCAACT
A: 5 1 0 0 5 5 0 0
C: 0 0 1 4 2 0 6 1
G: 1 1 6 3 0 1 0 0
T: 1 5 0 0 0 1 1 6


In [6]:
# Submission
solve('./txt/rosalind_cons.txt')

CAAAGAAATACACGTAACCGTTCCCCTACGGGCTATCGGACCTGGGACTGGTTATGCAGATCCCACTAGGTAGCTTGGCCGGCCAGTCAAGCAAACACATTACCTTTTTACCGAGCCGGTCAGACTTGCTTTAGACCAATTCTACGCATGTCTTTGTGATACCTACCATAACATTCAAACGAGACGCTTGTTTGTAGGCCCTAACCCTCTCCGCGACGGGGGTCACCATTGATCATCTTACGCGGGGCCACGTCAAGTATCCAAGGCAAAAACTTTAATAATCCAGCACAGAATTATTACTATACAAGTGGGTCGACGGTTCACTTTGGATAGGACAGGGGGACTAAGGTGTTGGACTCACGGCCGGCAGTAACAGTATCGAGCGTACTACGTGGCATGGATACCTGAGTTAGCGTACACTTTGGGAATGGCGATATAAATTACAGTTACTAGAACTTAATGACCCAGCCAGAGATCAGCGACACACTCGATCAGGTCACATTGCATCTGTTAGCTCAAACTCGATGGGGTTCGGCGCTACAAGACCCGAGTACTTGCTTCCTGGATTGCCATTAAAGAACCCAGCAGTCAAATAGCCGCGAGTTCTGATAGTGTCCGTTAGTGTGCGGTCCACAACGTTCCTCCGAATCTGGTGTGGAATTGACTGCCAGTCGGCTTTCTCGAGGTCTCGTCGGATCCTCCAACGACTCCGTTTGTGCCACTCCCGAGAGGTAGTTGCGTTGTCTTGATGGATGGCCCCGCGTACTGCCCGCCCCATATCCGTCCTCCTCACAGTTTGTTCAGCGGTGAGTCAAGCCGCTCCTCGTCATATGCCGGCGTCTTAATGAAGCTCCAACTAACGAACTGATCTTGAGTCAATTGGCCTCCCATCACCTTACACGGGAG
A: 2 4 3 3 2 3 4 5 2 5 1 5 2 1 3 4 3 3 2 1 4 3 3 2 2 2 1 5 2 3 2 2 2 2 3 1 0 1 2 5 2 0 1 3 2 